In [1]:
!pip uninstall serial
!pip install pyserial
!pip install getch
!python -m pip install persiantools

  Using cached pyserial-3.5-py2.py3-none-any.whl (90 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for getch: filename=getch-1.0-cp311-cp311-linux_x86_64.whl size=3893 sha256=dba7031a0e44b6f9d5d34ff6182766003991228b4e48fec08e90e11c7ee99196
  Stored in directory: /home/motalleb/.cache/pip/wheels/db/ce/6a/03a7bc3c9eefa3055674fa4b20c5357d3b3c16023d1ded86f5
Successfully built getch


In [2]:
import serial
import pandas as pd
from configparser import ConfigParser as cfgParser
from persiantools.jdatetime import JalaliDateTime as jd
from datetime import datetime
import time
import json
import requests
def delay(seconds: float):
    time.sleep(seconds)


configDir='./config/'
settingsPath=configDir+'settings.ini'

In [3]:
#loading information
settings=cfgParser()
settings.read(settingsPath)

dateTimeTemp = jd.now().isoformat()

# load output file path and user mapper
PLAIN_OUTPUT_CSV_FILE = settings.get(
    'Path', 'PLAIN_OUTPUT_CSV_FILE').replace('%dt', dateTimeTemp)
BACKUP_OUTPUT_CSV_FILE = settings.get(
    'Path', 'BACKUP_OUTPUT_CSV_FILE').replace('%dt', dateTimeTemp)
FINAL_OUTPUT_CSV_FILE = settings.get(
    'Path', 'FINAL_OUTPUT_CSV_FILE').replace('%dt', dateTimeTemp)
# FINAL_OUTPUT_XLSX_FILE = settings.get(
#     'Path', 'FINAL_OUTPUT_XLSX_FILE').replace('%dt', dateTimeTemp)
USERS_DICTIONARY_CSV_FILE = settings.get('Path', 'USERS_DICTIONARY_CSV_FILE')
usersMap = pd.read_csv(USERS_DICTIONARY_CSV_FILE)

# board info
SERIAL_PORT_NAME = settings.get('SerialConfig', 'SERIAL_PORT_NAME')
SERIAL_BAUD_RATE = settings.getint('SerialConfig', 'SERIAL_BAUD_RATE')

# commands
READ_FILE_COMMAND = settings.get('Commands', 'READ_FILE_COMMAND')
DELETE_FILE_COMMAND = settings.get('Commands', 'DELETE_FILE_COMMAND')
SET_TIME_COMMAND = settings.get('Commands', 'SET_TIME_COMMAND')

# output formatter settings
TEXT_FORMATTER_PREFIX = settings.get('Formatting', 'PREFIX_TEXT')
TEXT_FORMATTER_SUFFIX = settings.get('Formatting', 'SUFFIX_TEXT')
TEXT_FORMATTER_BOOT = settings.get('Formatting', 'BOOT_TEXT')
TEXT_FORMATTER_ERROR = settings.get('Formatting', 'ERROR_TEXT')
BASE_ENCODING = 'utf-8'


usersMap=pd.read_csv(USERS_DICTIONARY_CSV_FILE)

In [4]:

print('connecting to board')
SERIAL_PORT_NAME='/dev/ttyACM0'
serialClient = serial.Serial(SERIAL_PORT_NAME, SERIAL_BAUD_RATE)


def sendCommand(command):
    b = bytes(command, BASE_ENCODING)
    serialClient.write(b)

waitingForBoot = True
while(waitingForBoot):
    line = str(serialClient.readline().decode(BASE_ENCODING))
    if line.rfind(TEXT_FORMATTER_BOOT) >= 0:
        waitingForBoot = False
print('Boot Completed')
delay(1)

connecting to board
Boot Completed


In [5]:

print('sync time')
timeCommand = SET_TIME_COMMAND.replace('#year', str(datetime.now().year).rjust(4, '0')).replace('#month', str(datetime.now().month).rjust(2, '0')).replace('#day', str(datetime.now().day).rjust(2, '0')).replace('#hour', str(
    datetime.now().hour).rjust(2, '0')).replace('#minute', str(datetime.now().minute).rjust(2, '0')).replace('#seconds', str(datetime.now().second).rjust(2, '0')).replace('12OfWeek', str((datetime.now().weekday()+1) % 7+1))
sendCommand(timeCommand)
delay(1)

sync time


In [6]:

print('reading log file')
sendCommand(READ_FILE_COMMAND)
fileReader = True
logData = ''
while(fileReader):
    line = str(serialClient.readline().decode(BASE_ENCODING))
    logData += line
    print(line)
    if line.rfind(TEXT_FORMATTER_SUFFIX) >= 0:
        fileReader = False
    if line.rfind(TEXT_FORMATTER_ERROR) >= 0:
        raise Exception(
            "File is not found or there is error in log file reader method if you know there should be a file please restart the board otherwise use a tag on board and try again")
logData = logData.replace(TEXT_FORMATTER_BOOT, '')
logData = logData.replace(TEXT_FORMATTER_PREFIX, '')
logData = logData.replace(TEXT_FORMATTER_SUFFIX, '')
logData.replace('\r\n', '\n')
logData.replace('\r', '')
delay(1)


reading log file
LOGFILE_BEGIN

63513097,2023-09-20 08:40

63513097,2023-09-20 10:09

63513097,2023-09-20 17:04

63513097,2023-09-20 17:04

63513097,2023-09-21 10:07

396458C2,2023-09-21 10:26

63513097,2023-09-25 08:16

63513097,2023-09-25 08:32

63513097,2023-09-25 09:11

63513097,2023-09-26 08:06

63513097,2023-09-26 08:41

396458C2,2023-09-26 10:27

396458C2,2023-09-26 18:34

63513097,2023-09-27 08:24

396458C2,2023-09-27 10:52

396458C2,2023-09-27 18:46

63034997,2023-09-30 08:15

63513097,2023-09-30 08:33

63513097,2023-09-30 08:33

63513097,2023-09-30 08:33

63513097,2023-09-30 08:33

63513097,2023-10-01 08:04

396458C2,2023-10-01 08:41

63034997,2023-10-02 08:07

13488497,2023-10-02 08:07

13488497,2023-10-02 15:41

63034997,2023-10-02 17:02

63034997,2023-10-04 08:10

13488497,2023-10-04 10:27

63034997,2023-10-04 13:18

13488497,2023-10-04 17:01

396458C2,2023-10-04 19:14

B3FB3797,2023-10-04 19:21

63034997,2023-10-07 08:08

A3A23F97,2023-10-07 08:13

13488497,2023-10-07 08:

In [7]:
# import sys
# for i in sys.argv:
#     print('sending command : '+i)
#     sendCommand(i)
#     delay(1)
sendCommand(DELETE_FILE_COMMAND)

In [8]:
print('writing to backup')
f = open(BACKUP_OUTPUT_CSV_FILE, "a")
f.write(logData)
f.close()

writing to backup


In [9]:
def formatDateTime(input):
    date = datetime.fromisoformat(input)
    return (jd.to_jalali(date).isoformat().replace('T', ' '))

In [10]:

print('final formatting')

for index, row in usersMap.iterrows():
    logData = logData.replace(row['BOARD_ID'], row['USER_ID'])
logData = logData.replace('\n\n', '\n')
logData = logData.replace(',\n', '\n')

f = open(FINAL_OUTPUT_CSV_FILE, "w")
f.write('id,date_time'+logData)
f.close()
# data = pd.read_csv(PLAIN_OUTPUT_CSV_FILE)
# data.to_csv(FINAL_OUTPUT_CSV_FILE)

final formatting


In [ ]:
# def formatDateTime(input):
#     date=datetime.fromisoformat(input)
#     return (jd.to_jalali(date).isoformat())

In [ ]:

# reformat data

# for index, row in data.iterrows():
#     row['DATE_TIME'] = formatDateTime(row['DATE_TIME'])

# data.to_excel(FINAL_OUTPUT_XLSX_FILE)


In [11]:
dateTimeKey='date_time'
idKey='id'
presentKey='is_in'

In [12]:
data=pd.read_csv(FINAL_OUTPUT_CSV_FILE)
data.insert(len(data.columns), presentKey, 0)
data.head()

,id,date_time,is_in
0,ino0210,2023-09-20 08:40,0
1,ino0210,2023-09-20 10:09,0
2,ino0210,2023-09-20 17:04,0
3,ino0210,2023-09-20 17:04,0
4,ino0210,2023-09-21 10:07,0


In [19]:
def getKey(row):
    date=row[dateTimeKey].split(' ')[0]
    return date+row[idKey]
userIn=[]
for i in range(len(data)):
    item=data.iloc[i]
    
    if(getKey(item) not in userIn):
        data.iat[i, data.columns.get_loc(presentKey)] =1
        userIn.append(getKey(item))
    else:
        data.iat[i, data.columns.get_loc(presentKey)] =0
        userIn.remove(getKey(item))
data.head(20)
data.to_csv('test.csv')
jsonData= {
    'data':json.loads(data.to_json(orient ='records'))
}
# print(jsonData)

In [20]:
url = 'https://staff.hemend.com/api/attendance'
x = requests.post(url, json =jsonData)
print(x.text)

{"status_code":"OK"}
